# Module1: Face API のテスト

このノートブックでは、Face API を利用して、顔写真から顔の属性情報を取得します。
属性情報の推定に使用する画像ファイルはWeb上にあるものと、ローカルマシン上にあるものの両方を使用してテストします。

## Environment Setup
### 必要なライブラリをインストールする

各セルを1つずつ実行して、必要なライブラリをインストールします。

In [ ]:
!pip install azure-cognitiveservices-vision-face

互換性に関連するエラーは無視してください。 最後にこのメッセージが表示されている限り正常にインストールされています: Successfully installed azure-cognitiveservices-vision-face もしくは Requirement already satisfied

In [ ]:
import io
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageDraw
from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials

%matplotlib inline

## Task 1 - Face APIを使用したテスト

下のセルで、Face APIのキーを指定してセルを実行します。

In [ ]:
# サブスクリプションキーを記入します
subscription_key = '' #"<your_face_api_key>"
assert subscription_key

次のセルを実行して、Face APIのエンドポイントを指定します。

Face API サービスのインスタンスの Azure Portal からコピーしたエンドポイント値と一致するように、以下の face_endpoint の値を必ず更新してください。`値がスラッシュ（/）で終わっていることを確認してください`

In [ ]:
# face API のエンドポイントを記入します
face_endpoint = '' #"<your_face_api_endpoint>"

# 認証済みの FaceClient オブジェクトを作成します。
face_client = FaceClient(face_endpoint, CognitiveServicesCredentials(subscription_key))

## Face API の実行

まずはWeb上に既定で用意されている顔画像から顔属性を抽出してみましょう。

[FaceAttributes](https://docs.microsoft.com/rest/api/cognitiveservices/face/face/detectwithstream#faceattributes) をパラメータに指定することで、取得する属性を指定することができます。
その中で、Emotion プロパティは、0から1までの信頼度形式で感情を表し、anger, contempt, disgust, fear, happiness, neutral, sadness, surprise の8種類が推定可能です。


In [ ]:
# 画像ファイルのあるURLを指定
single_face_image_url = 'https://raw.githubusercontent.com/Microsoft/Cognitive-Face-Windows/master/Data/detection1.jpg'

# API コールで返したい属性、FaceAttributeType enum（文字列形式）のリスト
face_attributes = ['age', 'gender', 'headPose', 'smile', 'facialHair', 'glasses', 'emotion']

# 顔検出実行
detected_faces = face_client.face.detect_with_url(url=single_face_image_url, return_face_attributes=face_attributes)
if not detected_faces:
    raise Exception('画像から顔が検出されませんでした {}'.format(single_image_name))

'''
検出された顔を属性とバウンディングボックスで表示する
'''
# Face ID は、他の画像から検出された顔と比較するために使用される。
for face in detected_faces:
    print()
    print('Detected face ID from', ':')
    # ID of detected face
    print(face.face_id)
    # Show all facial attributes from the results
    print()
    print('Facial attributes detected:')
    print('Age:', face.face_attributes.age)
    print('Gender:', face.face_attributes.gender)
    print('Head pose:', face.face_attributes.head_pose)
    print('Smile:', face.face_attributes.smile)
    print('Facial hair:', face.face_attributes.facial_hair)
    print('Glasses:', face.face_attributes.glasses)
    print('Emotion:', face.face_attributes.emotion)
    print()

    
# 幅と高さを矩形内の座標に変換する
def getRectangle(faceDictionary):
    rect = faceDictionary.face_rectangle
    left = rect.left
    top = rect.top
    right = left + rect.width
    bottom = top + rect.height
    return ((left, top), (right, bottom))

# URLから画像をダウンロード
response = requests.get(single_face_image_url)
img = Image.open(BytesIO(response.content))

draw = ImageDraw.Draw(img)
for face in detected_faces:
    draw.rectangle(getRectangle(face), outline='red', width=3)

# 画像を表示
plt.figure(figsize=(8.0, 6.0))
#PIL image to matplotlib image
im_list = np.asarray(img)
plt.imshow(im_list)
plt.show()


## ストリームで検出

今度はローカルマシンから画像をストリームとしてアップロードする方式で顔検出を行います。

[Detect With Stream](https://docs.microsoft.com/rest/api/cognitiveservices/face/face/detectwithstream) を使用します。

emotions フォルダには、サンプルの顔写真が 01.jpg~12jpg まで用意してあります。

In [ ]:
# ローカルマシンの画像ファイルパスを指定
image_path = r"emotions/01.jpg"
image = open(image_path, 'r+b')

face_ids = []
# Detect faces
detected_faces = face_client.face.detect_with_stream(image, return_face_attributes=face_attributes)

if not detected_faces:
    raise Exception('画像から顔が検出されませんでした {}'.format(image_path))
    
for face in detected_faces:
    face_ids.append(face.face_id)

'''
検出された顔を属性とバウンディングボックスで表示する
'''
# Face ID は、他の画像から検出された顔と比較するために使用される。
for face in detected_faces:
    print()
    print('Detected face ID from', ':')
    # ID of detected face
    print(face.face_id)
    # Show all facial attributes from the results
    print()
    print('Facial attributes detected:')
    print('Age:', face.face_attributes.age)
    print('Gender:', face.face_attributes.gender)
    print('Head pose:', face.face_attributes.head_pose)
    print('Smile:', face.face_attributes.smile)
    print('Facial hair:', face.face_attributes.facial_hair)
    print('Glasses:', face.face_attributes.glasses)
    print('Emotion:', face.face_attributes.emotion)
    print()
    
# open the image
img = Image.open(image_path)

draw = ImageDraw.Draw(img)
for face in detected_faces:
    draw.rectangle(getRectangle(face), outline='red', width=3)

# 画像を表示
plt.figure(figsize=(8.0, 6.0))
#PIL image to matplotlib image
im_list = np.asarray(img)
plt.imshow(im_list)
plt.show()